In [59]:
import optuna
import numpy as np
import pandas as pd

from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [7]:
data=pd.read_csv(r'C:\Users\azad.aliyev\Downloads\data.csv')

In [164]:
sampled_data = data.sample(n=1000, random_state=42)

In [166]:
X = sampled_data.drop(columns=['anomaly'])
y = sampled_data['anomaly']

### Bayesian Optimization

In [168]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

In [75]:
baseline_metrics = {
    "Accuracy": 0.937,
    "Precision": 0.8212,
    "Recall": 0.7731,
    "F1 Score": 0.7965,
    "AUC": 0.8935,
}

In [77]:
# Define the objective function for Optuna
def objective(trial):
    # Hyperparameter suggestions
    hidden_units = trial.suggest_int("hidden_units", 16, 128, step=16)
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
    batch_size = trial.suggest_int("batch_size", 64, 256, step=64)
    activation = trial.suggest_categorical("activation", ["relu", "tanh"])
    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "rmsprop"])
    
    # Build the Autoencoder
    input_shape = X_train.shape[1]
    input_data = Input(shape=(input_shape,))
    encoded = Dense(hidden_units, activation=activation)(input_data)
    encoded = BatchNormalization()(encoded)
    encoded = Dropout(dropout_rate)(encoded)
    decoded = Dense(input_shape, activation="sigmoid")(encoded)
    autoencoder = Model(input_data, decoded)
    
    # Compile the model
    optimizer = Adam(learning_rate=learning_rate) if optimizer_name == "adam" else RMSprop(learning_rate=learning_rate)
    autoencoder.compile(optimizer=optimizer, loss="mean_squared_error")
    
    # Train the Autoencoder
    early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
    autoencoder.fit(
        X_train, X_train,
        epochs=50,
        batch_size=batch_size,
        validation_data=(X_test, X_test),
        shuffle=True,
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Predict and compute reconstruction errors
    y_pred = autoencoder.predict(X_test)
    mse = np.mean(np.power(X_test - y_pred, 2), axis=1)
    threshold = np.percentile(mse, 85)  # Use 85th percentile as the anomaly threshold
    y_pred_binary = (mse > threshold).astype(int)
    
    # Compute evaluation metrics
    precision = precision_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, mse)

    # Combine metrics (weighted average for optimization)
    return 0.4 * f1 + 0.3 * recall + 0.2 * precision + 0.1 * auc

In [79]:
# Run Optuna optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200, n_jobs=-1)

# Display the best hyperparameters
print("Best hyperparameters:", study.best_params)

[I 2024-12-01 17:08:35,926] A new study created in memory with name: no-name-5e38c1cf-858f-48a9-b7ee-47f9636d0e06
C:\Users\azad.aliyev\AppData\Local\Temp\ipykernel_17300\339139007.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step

[I 2024-12-01 17:08:46,207] Trial 11 finished with value: 0.5195027407612907 and parameters: {'hidden_units': 64, 'dropout_rate': 0.2980288304559988, 'learning_rate': 0.008579881340764473, 'batch_size': 256, 'activation': 'relu', 'optimizer': 'adam'}. Best is trial 11 with value: 0.5195027407612907.
[I 2024-12-01 17:08:46,227] Trial 7 finished with value: 0.269616739849298 and parameters: {'hidden_units': 32, 'dropout_rate': 0.40128650498271107, 'learning_rate': 0.008724482356087629, 'batch_size': 128, 'activation': 'tanh', 'optimizer': 'adam'}. Best is trial 11 with value: 0.5195027407612907.


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


C:\Users\azad.aliyev\AppData\Local\Temp\ipykernel_17300\339139007.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


[I 2024-12-01 17:15:01,072] Trial 197 finished with value: 0.5720467858634753 and parameters: {'hidden_units': 48, 'dropout_rate': 0.24541656164735173, 'learning_rate': 0.0008352247044346964, 'batch_size': 64, 'activation': 'tanh', 'optimizer': 'rmsprop'}. Best is trial 72 with value: 0.6720752298317277.


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


[I 2024-12-01 17:15:01,224] Trial 198 finished with value: 0.5660410269165399 and parameters: {'hidden_units': 32, 'dropout_rate': 0.24484044970386273, 'learning_rate': 0.000780175056508096, 'batch_size': 64, 'activation': 'relu', 'optimizer': 'rmsprop'}. Best is trial 72 with value: 0.6720752298317277.


Best hyperparameters: {'hidden_units': 96, 'dropout_rate': 0.3092770495947894, 'learning_rate': 0.0073218106416146345, 'batch_size': 64, 'activation': 'relu', 'optimizer': 'rmsprop'}


In [174]:
X = data.drop(columns=['anomaly'])
y = data['anomaly']

X_train = X.iloc[:650000, :]
X_test = X.iloc[650000:, :]
y_train = y.iloc[:650000]
y_test = y.iloc[650000:]

print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

Training data shape: (650000, 20)
Test data shape: (278898, 20)


In [135]:
# depth of autoencoder increased

input_shape = X_train.shape[1]
input_data = Input(shape=(input_shape,))

encoded = Dense(best_params['hidden_units'], activation=best_params['activation'])(input_data)
encoded = BatchNormalization()(encoded)
encoded = Dropout(best_params['dropout_rate'])(encoded)
encoded = Dense(best_params['hidden_units']//2, activation=best_params['activation'])(encoded)
encoded = BatchNormalization()(encoded)

decoded = Dense(best_params['hidden_units']//2, activation=best_params['activation'])(encoded)
decoded = BatchNormalization()(decoded)
decoded = Dense(best_params['hidden_units'], activation=best_params['activation'])(decoded)
decoded = BatchNormalization()(decoded)
decoded = Dense(input_shape, activation="sigmoid")(decoded)

best_model = Model(input_data, decoded)
best_model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), loss="mean_squared_error")

# Compile the model
#optimizer = Adam(learning_rate=best_params['learning_rate']) if optimizer_name == "adam" else RMSprop(learning_rate=best_params['learning_rate'])
#best_model.compile(optimizer="adam", loss="mean_absolute_error")

# Train the Autoencoder
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
best_model.fit(
    X_train, X_train,
    epochs=50,
    batch_size=best_params['batch_size'],
    validation_data=(X_test, X_test),
    shuffle=True,
    callbacks=[early_stopping],
    verbose=0
)

In [133]:
y_pred = best_model.predict(X_test)
mse = np.mean(np.power(X_test - y_pred, 2), axis=1)

threshold = np.percentile(mse, 85)
y_pred_binary = (mse > threshold).astype(int)

accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)
auc = roc_auc_score(y_test, mse)

print("Baseline Metrics:")
print(f"Accuracy: 0.937, Recall: 0.7731, Precision: 0.8212, F1 Score: 0.7965, AUC: 0.8935")

print("Optimized Model Metrics:")
print(f"Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1 Score: {f1:.4f}, AUC: {auc:.4f}")

8716/8716 ━━━━━━━━━━━━━━━━━━━━ 7s 743us/step
Baseline Metrics:
Accuracy: 0.937, Recall: 0.7731, Precision: 0.8212, F1 Score: 0.7965, AUC: 0.8935
Optimized Model Metrics:
Accuracy: 0.9369, Recall: 0.7727, Precision: 0.8207, F1 Score: 0.7960, AUC: 0.9134


In [117]:
best_params

{'hidden_units': 64,
 'dropout_rate': 0.4444672017557935,
 'learning_rate': 0.008962676579297325,
 'batch_size': 256,
 'activation': 'relu',
 'optimizer': 'rmsprop'}

### Hyperband

In [170]:
# Define the search space and the objective function
def objective(trial):
    # Hyperparameter suggestions
    hidden_units = trial.suggest_int("hidden_units", 16, 128, step=16)
    dropout_rate = trial.suggest_float("dropout_rate", 0.2, 0.5)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)
    batch_size = trial.suggest_int("batch_size", 64, 256, step=64)
    activation = trial.suggest_categorical("activation", ["relu", "tanh"])
    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "rmsprop"])

    # Build the Autoencoder
    input_shape = X_train.shape[1]
    input_data = Input(shape=(input_shape,))
    encoded = Dense(hidden_units, activation=activation)(input_data)
    encoded = BatchNormalization()(encoded)
    encoded = Dropout(dropout_rate)(encoded)
    decoded = Dense(input_shape, activation="sigmoid")(encoded)
    autoencoder = Model(input_data, decoded)

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate) if optimizer_name == "adam" else RMSprop(learning_rate=learning_rate)
    autoencoder.compile(optimizer=optimizer, loss="mean_squared_error")

    # Early stopping for regular training
    early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

    # Train the Autoencoder
    autoencoder.fit(
        X_train, X_train,
        epochs=50,  # Set max epochs; Hyperband will prune unpromising trials
        batch_size=batch_size,
        validation_data=(X_test, X_test),
        shuffle=True,
        callbacks=[early_stopping],
        verbose=0  # Suppress training output for better performance
    )

    # Predict and evaluate reconstruction errors
    y_pred = autoencoder.predict(X_test)
    mse = np.mean(np.power(X_test - y_pred, 2), axis=1)
    threshold = np.percentile(mse, 85)  # Use the 85th percentile as the anomaly threshold
    y_pred_binary = (mse > threshold).astype(int)

    # Compute evaluation metrics
    precision = precision_score(y_test, y_pred_binary)
    recall = recall_score(y_test, y_pred_binary)
    f1 = f1_score(y_test, y_pred_binary)
    auc = roc_auc_score(y_test, mse)
    
    # Compare with baseline
    if f1 < 0.75:  # Penalize if F1 Score is worse than 0.75
        return -1
    else:
        return 0.3 * f1 + 0.2 * recall + 0.2 * precision + 0.2 * auc + 0.1 * accuracy

In [172]:
# Create the study and run Hyperband optimization
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
    pruner=optuna.pruners.HyperbandPruner(min_resource=5, max_resource=50, reduction_factor=3),
)

study.optimize(objective, n_trials=150, timeout=3600, n_jobs=-1)  # Adjust timeout and trials as needed

# Display the best parameters and final results
print("Best hyperparameters:", study.best_params)
print("Best score:", study.best_value)

[I 2024-12-01 18:03:21,168] A new study created in memory with name: no-name-8af5474f-2454-459d-a7ed-9099768b1780
C:\Users\azad.aliyev\AppData\Local\Temp\ipykernel_17300\1127333341.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


[I 2024-12-01 18:03:30,894] Trial 7 finished with value: -1.0 and parameters: {'hidden_units': 112, 'dropout_rate': 0.3197190001781801, 'learning_rate': 0.006442740611297978, 'batch_size': 256, 'activation': 'relu', 'optimizer': 'adam'}. Best is trial 7 with value: -1.0.
C:\Users\azad.aliyev\AppData\Local\Temp\ipykernel_17300\1127333341.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


[I 2024-12-01 18:03:34,928] Trial 8 finished with value: -1.0 and parameters: {'hidden_units': 80, 'dropout_rate': 0.2697052679792505, 'learning_rate': 0.0034835547992661843, 'batch_size': 192, 'activation': 'relu', 'optimizer': 'rmsprop'}. Best is trial 7 with value: -1.0.
C:\Users\azad.aliyev\AppData\Local\Temp\ipykernel_17300\1127333341.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)


...

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


[I 2024-12-01 18:07:27,265] Trial 148 finished with value: -1.0 and parameters: {'hidden_units': 16, 'dropout_rate': 0.433234002670365, 'learning_rate': 0.006494042886918581, 'batch_size': 192, 'activation': 'tanh', 'optimizer': 'adam'}. Best is trial 7 with value: -1.0.


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


[I 2024-12-01 18:07:27,865] Trial 149 finished with value: -1.0 and parameters: {'hidden_units': 16, 'dropout_rate': 0.42525021506048355, 'learning_rate': 0.0034708944371325285, 'batch_size': 192, 'activation': 'tanh', 'optimizer': 'adam'}. Best is trial 7 with value: -1.0.


Best hyperparameters: {'hidden_units': 112, 'dropout_rate': 0.3197190001781801, 'learning_rate': 0.006442740611297978, 'batch_size': 256, 'activation': 'relu', 'optimizer': 'adam'}
Best score: -1.0


In [198]:
# depth of autoencoder increased

input_shape = X_train.shape[1]
input_data = Input(shape=(input_shape,))

encoded = Dense(best_params['hidden_units'], activation=best_params['activation'])(input_data)
encoded = BatchNormalization()(encoded)
encoded = Dropout(best_params['dropout_rate'])(encoded)
encoded = Dense(best_params['hidden_units']//2, activation=best_params['activation'])(encoded)
encoded = BatchNormalization()(encoded)

decoded = Dense(best_params['hidden_units']//2, activation=best_params['activation'])(encoded)
decoded = BatchNormalization()(decoded)
decoded = Dense(best_params['hidden_units'], activation=best_params['activation'])(decoded)
decoded = BatchNormalization()(decoded)
decoded = Dense(input_shape, activation="sigmoid")(decoded)

best_model = Model(input_data, decoded)
best_model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), loss="mean_squared_error")

# Compile the model
#optimizer = Adam(learning_rate=best_params['learning_rate']) if optimizer_name == "adam" else RMSprop(learning_rate=best_params['learning_rate'])
#best_model.compile(optimizer="adam", loss="mean_absolute_error")

# Train the Autoencoder
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
best_model.fit(
    X_train, X_train,
    epochs=50,
    batch_size=best_params['batch_size'],
    validation_data=(X_test, X_test),
    shuffle=True,
    callbacks=[early_stopping],
    verbose=0
)

In [206]:
y_pred = best_model.predict(X_test)
mse = np.mean(np.power(X_test - y_pred, 2), axis=1)

threshold = np.percentile(mse, 85)
y_pred_binary = (mse > threshold).astype(int)

accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)
auc = roc_auc_score(y_test, mse)

print("Baseline Metrics:")
print(f"Accuracy: 0.937, Recall: 0.7731, Precision: 0.8212, F1 Score: 0.7965, AUC: 0.8935")

print("Optimized Model Metrics:")
print(f"Accuracy: {accuracy:.4f}, Recall: {recall:.4f}, Precision: {precision:.4f}, F1 Score: {f1:.4f}, AUC: {auc:.4f}")

8716/8716 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step
Baseline Metrics:
Accuracy: 0.937, Recall: 0.7731, Precision: 0.8212, F1 Score: 0.7965, AUC: 0.8935
Optimized Model Metrics:
Accuracy: 0.9376, Recall: 0.7750, Precision: 0.8232, F1 Score: 0.7984, AUC: 0.8984
